In [1]:
%%capture
%matplotlib widget
#!pip install requests_cache

import matplotlib.pyplot as plt
import time
import requests
import pandas as pd
import numpy as np
import datetime
import math

import ipywidgets as widgets
from IPython.display import display, clear_output

import sys
sys.path.insert(1, '../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
sys.path.insert(1, '../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *

url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"

import os
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

In [2]:
# all widgets used in notebook
uploads = get_all_uploads(url, token, number_of_uploads=250)
upload_checkboxes = [widgets.Checkbox(value=False, description=upload.get('upload_name','--no-name--')) for upload in uploads]
scrollable_box = widgets.VBox(upload_checkboxes)
scrollable_area = widgets.Box([scrollable_box],
                              layout=widgets.Layout(
                                  overflow='scroll',
                                  border='1px solid gray',
                                  height='300px',
                                  width='600px',
                                  flex_flow='column',
                                  display='flex'
                              ))
select_uploads_button = widgets.Button(description='Confirm Selected Uploads', layout=widgets.Layout(width='600px',))
upload_selection_output = widgets.Output()

In [3]:
# widget logic
def save_selected_uploads(b):
    global upload_names, ca_data, cv_data
    upload_names = [cb.description for cb in upload_checkboxes if cb.value]
    with upload_selection_output:
        clear_output(wait=True)
        #print('Collecting CA and CV data from selected uploads. This can take a moment...')
        #ca_data = get_specific_entrytype_of_uploads(url, token, upload_names, 'CE_NOME_Chronoamperometry', with_meta=False)
        #cv_data = get_specific_entrytype_of_uploads(url, token, upload_names, 'CE_NOME_CyclicVoltammetry', with_meta=False)
        #clear_output(wait=True)
        print('Selected:', upload_names)
        #print('\nNumber of found CA files:', len(ca_data))
        #print('\nNumber of found CV files:', len(cv_data))

select_uploads_button.on_click(save_selected_uploads)

### Select Uploads

In [4]:
upload_names = []
display(scrollable_area, select_uploads_button, upload_selection_output)

Box(children=(VBox(children=(Checkbox(value=False, description='KMC3 tests'), Checkbox(value=False, descriptio…

Button(description='Confirm Selected Uploads', layout=Layout(width='600px'), style=ButtonStyle())

Output()

In [5]:
# TODO remove this in production
upload_names = ['test data for summer student project']

### Create Table from NOMAD entries

In [6]:

columns = ['sample_id', 'potential1 (mV vs RHE)', 'hold1 (s)', 'potential2 (mV vs RHE)', 'hold2 (s)', 'potential3 (mV vs RHE)', 'sweep speed (mV/s)', 'cycle (P2-P3)', 'duration (s)', 'duration (h)']
rows = []
hold1 = 120
hold2 = 0
sweep_speed = 100

# TODO from samples to uploads
# TODO get sample ids
for upload_name in upload_names:
    try:
        ca_data = get_specific_entrytype_of_uploads(url, token, [upload_name], 'CE_NOME_Chronoamperometry', with_meta=False)
        cv_data = get_specific_entrytype_of_uploads(url, token, [upload_name], 'CE_NOME_CyclicVoltammetry', with_meta=False)

        sample_id = ca_data[0].get('samples')
    
        # extract data from Chronoamperometry files
        rhe = ca_data[0].get('voltage_shift') or -0.933     #TODO get this from calibration files
        potential1 = ca_data[0].get('properties').get('step_1_potential') - rhe
    
        # extract data from CyclicVoltammetry files
        potential2 = cv_data[0].get('properties').get('limit_potential_1') # - rhe   # TODO is this the value we are looking for?
        potential3 = cv_data[0].get('properties').get('limit_potential_2') # - rhe   # TODO is this the value we are looking for?
     
        interval1 = hold1 + np.abs(potential1 - potential2)/sweep_speed
        interval2 = np.abs(potential2 - potential3)/sweep_speed + hold2
        cycles = min(100, math.floor((3600 - interval1) / interval2)) #TODO check min
        duration = interval1 + cycles*interval2
        # TODO maybe compare calculated cycles and duration with found data
        # cycle_number = len(ca_data)
    
        rows.append([sample_id, potential1*1000, hold1, potential2*1000, hold2, potential3*1000, sweep_speed, cycles, duration, duration/3600])
    
        print(upload_name, f': {len(cv_data)} CV files, {len(ca_data)} CA files')
    except:
        print(upload_name, "didnt work")

result = pd.DataFrame(rows, columns=columns)
result

test data for summer student project : 1 CV files, 1 CA files


,sample_id,potential1 (mV vs RHE),hold1 (s),potential2 (mV vs RHE),hold2 (s),potential3 (mV vs RHE),sweep speed (mV/s),cycle (P2-P3),duration (s),duration (h)
0,None,833.0,120,0.0,0,100.0,100,100,120.10833,0.033363


In [7]:
import datetime
date_now = datetime.datetime.now()
file_name = 'baybe_csv/nomad_result_table_ni_' + date_now.strftime("%Y%m%d") + '.csv'
result.to_csv(file_name, index=False)

In [8]:
rename_baybe = {
    'potential1 (mV vs RHE)': 'potential1',
    'hold1 (s)': 'hold1',
    'potential2 (mV vs RHE)': 'potential2',
    'hold2 (s)': 'hold2',
    'potential3 (mV vs RHE)': 'potential3',  
    'sweep speed (mV/s)': 'sweep_speed',
    'cycle (P2-P3)': 'cycles'}
baybe_table = result.rename(columns=rename_baybe)[['potential1', 'hold1', 'potential2', 'hold2', 'potential3', 'sweep_speed', 'cycles']]
import random 
baybe_table['Activity'] = range(1,len(result)+1) # TODO calculate this!!
baybe_table = baybe_table.replace('n/a', 0)    # this is from last year where sweep speed could be n/a TODO check if this is also possible this year
baybe_table.to_csv('baybe_csv/wateroxidation_ni.csv', index=False, header=True)
baybe_table

,potential1,hold1,potential2,hold2,potential3,sweep_speed,cycles,Activity
0,833.0,120,0.0,0,100.0,100,100,1


In [9]:
# TODO add calculation of targets
# criteria for evaluation:
# 1) min potential of CP at 500 mA/cm2, potential lower than untreated sample
# 2) min area under CV (should be equivalent to CP), area lower than untreated sample
# 3) min standard deviation of CP in 1)
# 4) min rms sum of (CV minus averaged CV)

# TODO add plots for parameter visualization

In [10]:
# TODO from samples to uploads
for upload_name in upload_names:
    try:
        cp_data = get_specific_entrytype_of_uploads(url, token, [upload_name], 'CE_NOME_Chronopotentiometry', with_meta=False)
        if len(cp_data) > 1:
            cp_data.sort(key=lambda x: datetime.datetime.strptime(x.get('datetime'), "%Y-%m-%dT%H:%M:%S.%f+00:00") if "." in x.get('datetime') else datetime.datetime.strptime(x.get('datetime'), "%Y-%m-%dT%H:%M:%S+00:00"))

            # TODO find CP at 500 mA/cm2
            cp_values = cp_data[0].get('voltage')
            min_cp = min(cp_values)
            min_std_cp = np.std(cp_values)
            print(min_std_cp)


        cv_data = get_specific_entrytype_of_uploads(url, token, [upload_name], 'CE_NOME_CyclicVoltammetry', with_meta=False)
        if len(cv_data) > 1:
            cv_data.sort(key=lambda x: datetime.datetime.strptime(x.get('datetime'), "%Y-%m-%dT%H:%M:%S.%f+00:00") if "." in x.get('datetime') else datetime.datetime.strptime(x.get('datetime'), "%Y-%m-%dT%H:%M:%S+00:00"))

        cv_values = cv_data[0].get('cycles', [{}])[0].get('current_density') # TODO use current or current density or ...
        #TODO calculate area
        cv_avg = np.mean(cv_values)
        rms_sum = np.sqrt(np.sum(np.square(cv_values - cv_avg)))
        print(rms_sum)

        print(upload_name)
    except:
        print(upload_name, "didnt work")

3.4064011165336687
test data for summer student project
